In [2]:
model_name = 'llama' # else 'gpt'

import getpass
import os
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from glob import glob
import json
from num2words import num2words
import pandas as pd

In [2]:
# UPDATE HOW THE ANSEWR IS PICKED

In [2]:
llama = ChatOllama(
        model="llama3.2",
        temperature=0,
        num_ctx=4096,
        num_predict=2048,
        verbose=True
    )
model = None
if model_name == 'llama':
    model = llama
else:
    if not os.environ.get("OPENAI_API_KEY"):
        os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
    model = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
        # base_url="...",
        # organization="...",
        # other params...
    )

In [29]:
files = glob('./selected_questions/*.jsonl')
questions = []
for path in files:
    question_type = path[path.rfind('/')+1:-6]
    with open(path, 'r') as file:
        for i in range(100):
            line = file.readline()
            question = json.loads(line)
            question['type'] = question_type
            questions.append(question)

In [30]:
import re

def extract_last_select_statement(query: str) -> str:
    # Remove the WITH clause and keep only the last SELECT statement
    query = re.sub(r'WITH\s+.*?\)\s*', '', query, flags=re.IGNORECASE | re.DOTALL)
    select_statements = re.findall(r'(SELECT\s+.*?FROM\s+.*?)(?=\s+WHERE|\s+GROUP BY|\s+ORDER BY|$)', query, re.IGNORECASE | re.DOTALL)
    return select_statements[-1].strip() if select_statements else ""

def extract_with_table_names(query: str) -> list:
    # Extract all table names defined in the WITH clause
    with_tables = re.findall(r'\b(\w+)\b\s+AS\s*\(', query, re.IGNORECASE)
    return with_tables

def extract_predicates(query: str, with_tables: list) -> str:
    # Find the last occurrence of WHERE and extract predicates after it
    where_matches = list(re.finditer(r'WHERE\s+', query, re.IGNORECASE))
    if not where_matches:
        return ""
    last_where = where_matches[-1].end()
    predicates_section = query[last_where:].strip()
    predicates_section = re.split(r'GROUP BY|ORDER BY', predicates_section, maxsplit=1, flags=re.IGNORECASE)[0].strip()
    
    # Split predicates by AND/OR while keeping the delimiters
    predicate_parts = re.split(r'\s+(AND|OR)\s+', predicates_section, flags=re.IGNORECASE)
    
    # List of spatial functions to remove
    spatial_functions = [
        'ST_Intersects', 'ST_Contains', 'ST_DWithin', 'ST_Touches', 'ST_Overlaps',
        'ST_Crosses', 'ST_Equals', 'ST_Disjoint', 'ST_Within'
    ]
    
    # Keep only non-spatial predicates and those not referring to WITH tables
    filtered_predicates = []
    i = 0
    while i < len(predicate_parts):
        predicate = predicate_parts[i].strip()
        if not any(func in predicate for func in spatial_functions) and not any(tbl in predicate for tbl in with_tables):
            filtered_predicates.append(predicate)
        if i + 1 < len(predicate_parts):
            filtered_predicates.append(predicate_parts[i + 1])  # Keep AND/OR operators
        i += 2
    
    return " ".join(filtered_predicates).strip()

def rebuild_query(original_query: str, q_type: str) -> str:
    with_tables = extract_with_table_names(original_query)
    select_statement = extract_last_select_statement(original_query)
    non_spatial_predicates = extract_predicates(original_query, with_tables)
    if 'loc' in q_type:
        non_spatial_predicates += ' AND addr_city IS NOT Null'
    # Extract the table names from the FROM clause
    from_match = re.search(r'FROM\s+(.*?)(WHERE|GROUP BY|ORDER BY|$)', select_statement, re.IGNORECASE | re.DOTALL)
    if from_match:
        tables = [tbl.strip() for tbl in from_match.group(1).split(',')]
        remaining_tables = [tbl for tbl in tables if tbl not in with_tables]
        if remaining_tables:
            select_statement = re.sub(r'FROM\s+.*?(WHERE|GROUP BY|ORDER BY|$)', f'FROM {remaining_tables[0]} \1', select_statement, flags=re.IGNORECASE)
    
    # Clean up any remaining commas in FROM clause
    # select_statement = re.sub(r',\s*', ' ', select_statement)
    
    new_query = select_statement.strip()
    if non_spatial_predicates:
        new_query += f" WHERE {non_spatial_predicates}"
    
    new_query += " ORDER BY RANDOM() LIMIT 1"
    return new_query

In [3]:
import random

def random_angle():
    direction_filters = [
        ['north', random.choice([random.uniform(0.0,22.5), random.uniform(337.5,360)])],
        ['northeast', random.uniform(22.5, 67.5)],
        ['east', random.uniform(67.5, 112.5)],
        ['southeast', random.uniform(112.5, 157.5)],
        ['south', random.uniform(157.5, 202.5)],
        ['southwest', random.uniform(202.5, 247.5)],
        ['west', random.uniform(247.5, 292.5)],
        ['northwest', random.uniform(292.5, 337.5)]
    ]

    d = random.choice(range(len(direction_filters)))    
    # Select a random value from the combined list
    # random_angle_in_direction = int(random.choice(direction_filters[d][1]))
    return {'angle_description': direction_filters[d][0], 'angle': direction_filters[d][1]}
random_angle()

{'angle_description': 'southeast', 'angle': 151.82904794657867}

In [15]:
system_prompt2 = str("Given a question and a text answer, parse the text answer to json format."
                     " The location must be provided as a complete address,"
                     " any measurment must be in metric units,"
                     " and directions must be converted to azimuth angle in degress."
                     " Try to match the following schema:"
                     """
                        {
                            "name" string
                            "address": string,
                            "count": integer,
                            "distance": integer,
                            "length": integer,
                            "area": integer,
                            "azimuth_angle": integer,
                            %OTHER_ATT%
                        }
                    If a value is missing don't include it in the output, and don't write any comments.
                    All json blocks must be enclosed with ```json and ```
                     """)

In [16]:
import psycopg
from psycopg.rows import dict_row

def run_sql(sql, conn, timeout):
    cur = conn.cursor()
    cur.execute("SET statement_timeout = %d" % timeout)
    try:
        cur.execute(sql)
    except Exception as e:
        conn.rollback()
        return {'output': [], 'error': str(e)}
    records = cur.fetchmany(size=100)
    cur.close()
    return {'output': [{k: row[k] for k in row if row[k] is not None} for row in records], 'error': ''}
conn = psycopg.connect(
    host = 'localhost',
    dbname = 'osm_ca',
    user = 'postgres',
    password = 'postgres',
    port = 5432,
    row_factory=dict_row,
    # options="-c statement_timeout=180000"
)


In [17]:
# this function was generated with ChatGPT to get random answers for the multihop quesitons for the random baseline

from faker import Faker

fake = Faker()


def get_random_value(attribute: str):
    attribute_generators = {
        "Architect": fake.name,
        "Built": lambda: random.randint(1800, 2025),
        "Created": lambda: fake.date(),
        "Established": lambda: random.randint(1700, 2025),
        "Director": fake.name,
        "Founder": fake.name,
        "Headquarters": fake.city,
        "Opened": lambda: random.randint(1800, 2025),
        "Opening date": fake.date,
        "Affiliated university": lambda: f"{fake.company()} University",
        "Emergency department": lambda: random.choice(["Yes", "No"]),
        "Helipad": lambda: random.choice(["Yes", "No"]),
        "Date opened": fake.date,
        "Volume of largest tank": lambda: f"{random.randint(1000, 100000)} liters",
        "Capacity": lambda: random.randint(50, 100000),
        "Former names": lambda: fake.company(),
        "Motto": fake.sentence,
        "Mascot": lambda: fake.word().capitalize(),
        "Nickname": lambda: fake.word().capitalize(),
        "Designed by": fake.name,
        "Nearest\xa0city": fake.city
    }
    generator = attribute_generators.get(attribute)
    if generator:
        return generator()
    else:
        return "Attribute not recognized"


In [62]:
import random
import importlib
import evaluate
importlib.reload(evaluate)
from tqdm import tqdm

q_types = set([q['type'] for q in questions])
q_answers = {t: [{'answers': q['answers'], 'id': q['id']} for q in questions if q['type'] == t] for t in q_types}
# modify it to query and select a random answer using sql
answers = []
for q in tqdm(questions):
    element = []
    if 'name' in q['type'] or 'loc' in q['type']:
        sql = rebuild_query(q['sql'], q['type'])
        element = run_sql(sql.replace('\x01', ''), conn, 1000)['output']
    elif 'angle' in q['type']:
        element = [random_angle()]
    elif 'area' in q['type']:
        element = [{'area': random.randint(1,10**10)}]
    elif 'length' in q['type']:
        element = [{'length': random.randint(1,10**10)}]
    elif 'distance' in q['type']:
        element = [{'distance': random.randint(1,10**10)}]
    elif 'count' in q['type']:
        element = [{'count': random.randint(1,10**10)}]
    elif 'multihop1' in q['type']:
        att = q['answers'][0]['multihop_attribute']
        element = [{'multihop_long_answer': att + ' ' + str(get_random_value(att))}]
    key = ''
    if 'multihop1' in q['type']:
        key = 'multihop_long_answer'
    elif 'name' in q['type']:
        key = 'name'
    elif 'loc' in q['type']:
        key = 'address'
    elif 'angle' in q['type']:
        key = 'angle_description'
    elif 'area' in q['type']:
        key = 'area'
    elif 'length' in q['type']:
        key = 'length'
    elif 'count' in q['type']:
        key = 'count'
    elif 'distance' in q['type']:
        key = 'distance'
    true_answer = []
    for a in element:
        v = evaluate.get_osm_value(a, key)
        if v == None:
            continue
        if key in ['area', 'length', 'count', 'distance']:
            v = num2words(v)
        if 'area' == key:
            v += ' meters squared'
        elif key in ['length', 'distance']:
            v += ' meters'
        true_answer.append(v)
    _true_answer = '\n'
    i = 0
    for t in true_answer:
        _true_answer += '\n' + t
        i+=1
        if len(_true_answer) > 256 or i >= 10:
            break
    _true_answer = _true_answer[1:]
    answers.append({'id': q['id'], 'content': _true_answer})


[nltk_data] Downloading package punkt to /Users/majid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/majid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 2800/2800 [04:55<00:00,  9.47it/s] 


In [64]:
answered = {q['id']: q for q in json_answers}

TypeError: 'AIMessage' object is not subscriptable

In [68]:
json_answers = []

for i in tqdm(range(len(questions))):
    q = questions[i]
    if 'multihop1' in q['type']:
        sys_prompt= system_prompt2.replace('%OTHER_ATT%', '"%s": string' % q['answers'][0]['multihop_attribute'])
    else:
        sys_prompt= system_prompt2.replace('%OTHER_ATT%', '')
    a = answers[i]
    if a['id'] != q['id']:
        for j in range(len(answers)):
            if answers[j]['id'] == q['id']:
                a = answers[j]
                break
    messages = [
            SystemMessage(content=sys_prompt),
            HumanMessage(content="Question: %s\nAnswer: %s" % (q['question'], a['content']))
        ]
    json_answers.append({'content': llama.invoke(messages).content, 'id': q['id']})

100%|██████████| 2800/2800 [1:01:37<00:00,  1.32s/it]  


In [31]:
# with open('./json_answers_shuffled.json', 'w') as file:
#         file.write(json.dumps(json_answers, indent=2))
        # file.write(json.dumps([{'content': json_answers[i].content, 'id': questions[i]['id']} for i in range(len(questions))], indent=2))
with open('./json_answers_shuffled.json', 'r') as file:
        json_answers = json.loads(file.read())

In [34]:
import re

def flatten_if_nested(array):
    # Check if the input is a list and contains nested lists
    if isinstance(array, list) and any(isinstance(item, list) for item in array):
        flattened = []
        for item in array:
            if isinstance(item, list):
                flattened.extend(flatten_if_nested(item))
            else:
                flattened.append(item)
        return flattened
    else:
        return array  # Return the input as-is if it's not a list or doesn't contain nested lists

def extract_json_blocks(text, i):
    # Regular expression pattern to match JSON blocks
    pattern = r'```[\s]*json(.*?)```'
    pattern1 = r'\b\d+(?:_\d+)*\b'
    pattern2 = r'\b\d+(?:,\d+)*\b'
    pattern3 = r'//.*?\n'
    pattern4 = r',\s*}'
    pattern5 = r'}\s*{'
    # Find all JSON blocks
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Parse each match to ensure valid JSON
    json_blocks = []
    for match in matches:
        try:
            # Remove any leading/trailing whitespace and parse as JSON
            s = match.strip()
            s = re.sub(pattern1, lambda x: x.group().replace('_', ''), s)
            s = re.sub(pattern2, lambda x: x.group().replace(',', ''), s)
            s = re.sub(pattern3, '', s)
            s = re.sub(pattern4, '}', s)
            s = s.replace('''\\\'''', '''\'''').replace('''\\&''', '''&''').replace(': integer', ': null').replace(' * ', '')
            # Case 1: Replace any number followed by '+' with just the first number
            s = re.sub(r'(\d[\d\s]*)(?:\s*\+\s*[\d\s\+]+)', lambda m: m.group(1).replace(' ', ''), s)
            # Case 2: Remove spaces within standalone numbers
            s = re.sub(r'\b\d[\d\s]*\b', lambda m: m.group(0).replace(' ', ''), s)
            if re.search(pattern5, s):
                s = re.sub(pattern5, '},\n{', s)
                s = '[\n%s\n]' % s
            convert_area = False
            if 'acres' in s:
                convert_area = True
                s = s.replace(' acres,', ',')
                s = s.replace('acres,', ',')
            json_data = json.loads(s)
            if convert_area and 'area' in json_data:
                json_data['area'] = json_data['area'] * 4046.8564224
            json_blocks.append(json_data)
        except json.JSONDecodeError as w:
            print(w)
            # If parsing fails, print an error message (can log or handle as needed)
            print(i)
            print(s)
            print("Warning: Found an invalid JSON block.") 
    return flatten_if_nested(json_blocks)

In [35]:
parsed_answers = []
for q in questions:
    a = json_answers[i]
    if a['id'] != q['id']:
        for j in range(len(json_answers)):
            if json_answers[j]['id'] == q['id']:
                a = json_answers[j]
                break
    parsed_answers.append(extract_json_blocks(a['content'], i))

In [36]:
from geopy.geocoders import Nominatim
from pyproj import Geod

geod = Geod(ellps='WGS84')
geocoder = Nominatim(user_agent="Geocoder")

In [5]:
import importlib
import evaluate
importlib.reload(evaluate)
import numpy as np

[nltk_data] Downloading package punkt to /Users/majid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/majid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/majid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/majid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [79]:

text_evaluation = []

# evaluate text answers

for i in range(len(questions)):
    q = questions[i]
    a = answers[i]
    if a['id'] != q['id']:
        for j in range(len(answers)):
            if answers[j]['id'] == q['id']:
                a = answers[j]
                break
    text_answer = a['content']
    key = ''
    if 'multihop1' in q['type']:
        key = 'multihop_long_answer'
    elif 'name' in q['type']:        
        key = 'name'
    elif 'loc' in q['type']:
        key = 'address'
    elif 'angle' in q['type']:
        key = 'angle_description'
    elif 'area' in q['type']:
        key = 'area'
    elif 'length' in q['type']:
        key = 'length'
    elif 'count' in q['type']:
        key = 'count'
    elif 'distance' in q['type']:
        key = 'distance'
    true_answer = []
    for a in q['answers']:
        v = evaluate.get_osm_value(a, key)
        if v == None:
            continue
        if key in ['area', 'length', 'count', 'distance']:
            v = num2words(v)
        if 'area' == key:
            v += ' meters squared'
        elif key in ['length', 'distance']:
            v += ' meters'
        true_answer.append(v)
    if len(text_answer):
        true_answer = '\n'.join(true_answer)
        P, R, F1 = evaluate.evaluate_entity_names(text_answer, true_answer)
        text_evaluation.append({'attempted': True, 'P': P, 'R': R, 'F1': F1})
    else:
        text_evaluation.append({'attempted': False, 'P': 0, 'R': 0, 'F1': 0})
    

In [80]:
df = pd.DataFrame(text_evaluation)
df['type'] = [q['type'] for q in questions]
df['id'] = [q['id'] for q in questions]
df.to_csv(f'./shuffled_text_eval.csv', index=False)

In [39]:
import tqdm

In [40]:
# evaluate parsed_answers
parsed_evaluation = []
progress = tqdm.tqdm(range(len(questions)))
def imporved_f1(new_f1, scores):
    return ('F1' not in scores) or (new_f1 > scores['F1'])

for i in progress:
    q = questions[i]
    parsed_answer = parsed_answers[i]
    scores = {'attempted': False}
    if 'multihop1' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'multihop_answer')
            if v == None:
                continue
            for p in parsed_answer:
                pred_answer = p.get(a['multihop_attribute'], None)
                if pred_answer == None or len(pred_answer) == 0:
                    continue
                P, R, F1 = evaluate.evaluate_entity_names(pred_answer, v)
                if imporved_f1(F1, scores):
                    scores = {'attempted': True, 'P': P, 'R': R, 'F1': F1}
    elif 'name' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'name')
            if v == None:
                continue
            for p in parsed_answer:
                pred_answer = p.get('name', None)
                if pred_answer == None or len(pred_answer) == 0:
                    continue
                P, R, F1 = evaluate.evaluate_entity_names(pred_answer, v)
                if imporved_f1(F1, scores):
                    scores = { 'attempted': True, 'P': P, 'R': R, 'F1': F1}
    elif 'loc' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'address')
            loc = evaluate.get_osm_value(a, 'location')
            if v == None:
                continue
            for p in parsed_answer:
                pred_answer = p.get('address', None)
                if pred_answer == None or len(pred_answer) == 0:
                    continue
                if type(pred_answer) == type([]):
                    pred_answer = ', '.join(pred_answer)
                if type(pred_answer) == dict:
                    pred_answer = ', '.join(pred_answer.values())
                P, R, F1 = evaluate.evaluate_entity_names(pred_answer, v)
                if imporved_f1(F1, scores):
                    scores.update({'attempted': True,'P': P, 'R': R, 'F1': F1})
                pred_loc = evaluate.get_location_by_address(geocoder, pred_answer)
                if pred_loc == None:
                    continue
                distance_error = evaluate.evaluate_location(geod, [pred_loc], [loc])[0]
                distance_limit = 5*10**5
                if distance_error > distance_limit:
                    distance_error = 1.0
                else:
                    distance_error /= distance_limit
                if distance_error < scores.get('distance_error', float('inf')):
                    scores['distance_error'] = distance_error
    elif 'angle' in q['type']:
        for a in q['answers']:
            angle = evaluate.get_osm_value(a, 'angle')
            angle_desc = evaluate.get_osm_value(a, 'angle_description')
            if angle == None:
                continue
            for p in parsed_answer:
                pred_angle = p.get('azimuth_angle', None)
                try:
                    pred_angle = int(pred_angle)
                except:
                    continue
                pred_answer = evaluate.get_angle_desc(pred_angle)
                if pred_answer == None or len(pred_answer) == 0:
                    continue
                P, R, F1 = evaluate.evaluate_entity_names(pred_answer, angle_desc)
                if imporved_f1(F1, scores):
                    scores.update({'attempted': True,'P': P, 'R': R, 'F1': F1})
                angle_error = evaluate.evaluate_angle([pred_angle], [angle])[0]
                if angle_error < scores.get('angle_error', float('inf')):
                    scores['angle_error'] = angle_error
    elif 'area' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'area')
            if v == None:
                continue
            for p in parsed_answer:
                pred_v = p.get('area', None)
                if pred_v == None:
                    continue
                try:
                    pred_v = int(pred_v)
                except:
                    continue
                relative_error = evaluate.evaluate_measurement(pred_v, v)
                if relative_error < scores.get('relative_error', float('inf')):
                    scores['relative_error'] = relative_error
                    scores['attempted'] = True
    elif 'length' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'length')
            if v == None:
                continue
            for p in parsed_answer:
                pred_v = p.get('length', None)
                if pred_v == None:
                    continue
                try:
                    pred_v = int(pred_v)
                except:
                    continue
                relative_error = evaluate.evaluate_measurement(pred_v, v)
                if relative_error < scores.get('relative_error', float('inf')):
                    scores['relative_error'] = relative_error
                    scores['attempted'] = True
    elif 'count' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'count')
            if v == None:
                continue
            for p in parsed_answer:
                pred_v = p.get('count', None)
                if pred_v == None:
                    continue
                try:
                    pred_v = int(pred_v)
                except:
                    continue
                relative_error = evaluate.evaluate_measurement(pred_v, v)
                if relative_error < scores.get('relative_error', float('inf')):
                    scores['relative_error'] = relative_error
                    scores['attempted'] = True
    elif 'distance' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'distance')
            if v == None:
                continue
            for p in parsed_answer:
                pred_v = p.get('distance', None)
                if pred_v == None:
                    continue
                try:
                    pred_v = int(pred_v)
                except:
                    continue
                relative_error = evaluate.evaluate_measurement(pred_v, v)
                if relative_error < scores.get('relative_error', float('inf')):
                    scores['relative_error'] = relative_error
                    scores['attempted'] = True
    parsed_evaluation.append(scores)

100%|██████████| 2800/2800 [00:02<00:00, 1060.11it/s]


In [41]:
df = pd.DataFrame(parsed_evaluation)
df['type'] = [q['type'] for q in questions]
df['id'] = [q['id'] for q in questions]
df.loc[df['P'].isna(), 'P'] = 0
df.loc[df['R'].isna(), 'R'] = 0
df.loc[df['F1'].isna(), 'F1'] = 0
df.loc[df['distance_error'].isna(), 'distance_error'] = 1.0
df.loc[df['angle_error'].isna(), 'angle_error'] = 1.0
df.loc[df['relative_error'].isna(), 'relative_error'] = 1.0

df.to_csv(f'./shuffled_parsed_eval.csv', index=False)

In [77]:
# one correct answer vs. one predicted answer
simulate_angle_error = [i for i in range(361)]
errors = []
for a1 in range(100): # a1 is correct answer
    a1 = random.choice(range(361))
    for _ in range(1): # repetitions for each correct answer
        a2 = random_angle()['angle'] #random.choice(range(361))
        errors += evaluate.evaluate_angle([a1], [a2])
print(np.mean(errors))

0.5150408509020873


In [30]:
# one correct answer vs. N predicted answers
N = 3
simulate_angle_error = [i for i in range(361)]
errors = []
for a1 in range(361): # a1 is correct answer
    for _ in range(1): # repetitions for each correct answer
        _errors = []
        for i in range(N): # generate multiple predictions
            a2 = random.choice(range(361)) #random_angle()['angle']
            _errors += evaluate.evaluate_angle([a1], [a2])
        errors += [min(_errors)]    
print(np.mean(errors))

0.26577408433364114
